In [5]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from collections import defaultdict
import re
import sqlite

Plan: walk through files
grab: title, ingredients(main), ingredients(detailed), old url, short description
convert dict to df and then write to SQLlite instance

TODO: 
get ingredients, instructions, 


In [58]:
pwd

'/Users/wolfiedieterich/Desktop/america_eats'

In [59]:
cd old_html/

/Users/wolfiedieterich/Desktop/america_eats/old_html


In [10]:
with open("Classic Beef Brisket With Caramelized Onions.html") as fo:
    soup = bs(fo.read(), 'html.parser')

In [38]:
soup

<div class="grid-wrap">
<article class="card static noclickarea recipe-detail-card recipe-has-topnote recipe-has-media recipe-has-horizontal-media recipe-has-horizontal-media recipe-has-time-yield" data-analytics-category="Recipe Detail Full" data-id="1018922" data-seo-image-url="https://static01.nyt.com/images/2017/09/13/dining/13ROSHRECIPE/13ROSHRECIPE-videoSixteenByNineJumbo1600.jpg" data-type="recipe" data-url="/recipes/1018922-swiss-chard-and-lamb-torte-with-fennel-pomegranate-relish">
<header>
<div class="title-container">
<h1 class="recipe-title title name" data-name="Swiss Chard and Lamb Torte With Fennel-Pomegranate Relish" data-share-title="Swiss+Chard+and+Lamb+Torte+With+Fennel-Pomegranate+Relish+by+Joan+Nathan" data-url="/recipes/1018922-swiss-chard-and-lamb-torte-with-fennel-pomegranate-relish" itemprop="name">
          Swiss Chard and Lamb Torte With Fennel-Pomegranate Relish
        </h1>
</div>
<div class="recipe-subhead">
<h3 class="byline ">
<a class="author personal

In [13]:
def get_instructions(soup):
    l = [i.get_text().replace('Â', '') for i in soup.find(class_='recipe-steps').find_all('li')]
    try:
        l.extend([i.get_text().replace('Â', '') for i in soup.find(class_='recipe-notes').find_all('li')])
    except AttributeError as e:
        pass
    return l

In [14]:
def get_ingredients(soup): 
    ingredients = soup.find_all(itemprop='recipeIngredient')
    l = []
    for i in ingredients:
        l.append((i.find(class_='quantity').text.replace('Â',''), i.find(class_='ingredient-name').text))
    return l

In [15]:
get_ingredients(soup)

[('3', 'tablespoons kosher salt'),
 ('½', 'teaspoon black pepper'),
 ('1', 'teaspoon paprika (preferably Hungarian)'),
 ('', 'Pinch of cayenne'),
 ('5 to 7', 'pounds beef brisket, not too lean'),
 ('1', 'cup red or white wine'),
 ('12', 'cloves'),
 ('1', 'whole head of garlic'),
 ('3', 'bay leaves'),
 ('3', 'allspice berries'),
 ('6', 'large onions, peeled, and sliced crosswise 1/4-inch thick'),
 ('3', 'tablespoons olive oil'),
 ('', 'Parsley sprigs, for garnish'),
 ('¼', 'cup slivered scallions (optional)')]

In [27]:
def get_desc_tags(soup):
    l = []
    for i, child in enumerate(soup.find(class_='special-diets tag-block').children):
        contents = child.string.rstrip().lstrip()
        if re.search('[a-zA-Z]', child.string):
            l.append(child.string.rstrip().lstrip())
    return tuple(l)

In [30]:
file_list = [file for file in os.listdir() if '.html' in file]

In [31]:
file_list

['Classic Beef Brisket With Caramelized Onions.html',
 'Fresh Fig Cake With Honey Cream-Cheese Frosting.html',
 'Swiss Chard and Lamb Torte With Fennel-Pomegranate Relish.html']

In [74]:
d = defaultdict(dict)
for file in file_list:
    with open(file, 'r') as fo:
        soup = bs(fo.read(), 'html.parser')
        d[file]['h1'] = soup.h1.get_text().lstrip().rstrip()
        d[file]['desc_tags'] = str(get_desc_tags(soup))
        d[file]['ingredients'] = str(get_ingredients(soup))
        d[file]['instructions'] = str(get_instructions(soup))
        d[file]['yield'] = soup.find(itemprop='recipeYield').get_text()
        d[file]['description'] = soup.find(itemprop='description').p.get_text()

In [75]:
d

defaultdict(dict,
            {'Classic Beef Brisket With Caramelized Onions.html': {'desc_tags': "('Jewish', 'Allspice', 'Beef Brisket', 'Onion')",
              'description': 'This is a classic brisket recipe with no bells and whistles, just deep flavor, moist succulent meat and lots of caramelized onions. The only caveat: Buy a brisket that’s not too lean. You want it well-marbled with fat or the result will be dry, not juicy.',
              'h1': 'Classic Beef Brisket With Caramelized Onions',
              'ingredients': "[('3', 'tablespoons kosher salt'), ('½', 'teaspoon black pepper'), ('1', 'teaspoon paprika (preferably Hungarian)'), ('', 'Pinch of cayenne'), ('5 to 7', 'pounds beef brisket, not too lean'), ('1', 'cup red or white wine'), ('12', 'cloves'), ('1', 'whole head of garlic'), ('3', 'bay leaves'), ('3', 'allspice berries'), ('6', 'large onions, peeled, and sliced crosswise 1/4-inch thick'), ('3', 'tablespoons olive oil'), ('', 'Parsley sprigs, for garnish'), ('¼', '

In [76]:
df = pd.DataFrame.from_dict(d)

In [77]:
df = df.T

In [78]:
df = df.rename_axis('name')

In [79]:
df

,desc_tags,description,h1,ingredients,instructions,yield
name,,,,,,
Classic Beef Brisket With Caramelized Onions.html,"('Jewish', 'Allspice', 'Beef Brisket', 'Onion')",This is a classic brisket recipe with no bells...,Classic Beef Brisket With Caramelized Onions,"[('3', 'tablespoons kosher salt'), ('½', 'teas...","['In a small bowl, mix together salt, pepper, ...",6 to 8 servings
Fresh Fig Cake With Honey Cream-Cheese Frosting.html,"('Cream Cheese', 'Fig', 'Honey', 'Jam')","This dense and deeply figgy cake, adapted from...",Fresh Fig Cake With Honey Cream-Cheese Frosting,"[('', 'Butter, for greasing the pan'), ('3', '...",['Make the cake: Heat oven to 325 degrees. But...,12 servings
Swiss Chard and Lamb Torte With Fennel-Pomegranate Relish.html,"('Jewish', 'Middle Eastern', 'Fennel', 'Lamb',...",Festive dishes in Israel and throughout the Mi...,Swiss Chard and Lamb Torte With Fennel-Pomegra...,"[('2 ⅓', 'cups medium-grain or sushi rice (1 p...",['Prepare the torte: Place rice in a heat-proo...,10 to 12 servings


In [93]:
import sqlite3

In [94]:
db = sqlite3.connect('am_v1.sqlite')

In [95]:
cursor = db.cursor()

In [96]:
cursor.execute("""drop table Recipes""")

In [97]:
db.commit()

In [98]:
cursor.execute("""create table Recipes (name text, desc_tags text, description text, h1 text, ingredients text,
                instructions text, yield text)""")

In [99]:
db.commit()

In [100]:
df

,desc_tags,description,h1,ingredients,instructions,yield
name,,,,,,
Classic Beef Brisket With Caramelized Onions.html,"('Jewish', 'Allspice', 'Beef Brisket', 'Onion')",This is a classic brisket recipe with no bells...,Classic Beef Brisket With Caramelized Onions,"[('3', 'tablespoons kosher salt'), ('½', 'teas...","['In a small bowl, mix together salt, pepper, ...",6 to 8 servings
Fresh Fig Cake With Honey Cream-Cheese Frosting.html,"('Cream Cheese', 'Fig', 'Honey', 'Jam')","This dense and deeply figgy cake, adapted from...",Fresh Fig Cake With Honey Cream-Cheese Frosting,"[('', 'Butter, for greasing the pan'), ('3', '...",['Make the cake: Heat oven to 325 degrees. But...,12 servings
Swiss Chard and Lamb Torte With Fennel-Pomegranate Relish.html,"('Jewish', 'Middle Eastern', 'Fennel', 'Lamb',...",Festive dishes in Israel and throughout the Mi...,Swiss Chard and Lamb Torte With Fennel-Pomegra...,"[('2 ⅓', 'cups medium-grain or sushi rice (1 p...",['Prepare the torte: Place rice in a heat-proo...,10 to 12 servings


In [81]:
df.to_sql('Recipes', conn, if_exists='append')

In [86]:
def query_db(page_name):
    response = cursor.execute(f"select * from Recipes")
    print(response.fetchall())

In [87]:
query_db('Classic Beef Brisket With Caramelized Onions.html')

[]


In [89]:
response = cursor.execute('Select * from Recipes')

In [90]:
df1 = pd.DataFrame(response.fetchall())

In [91]:
df1

""
